In [100]:
import sys
import math
import numpy as np
import pandas
import requests
import shutil
from fitparse import FitFile
import mediapipe as mp
import cv2
import trimesh
from trimesh.transformations import rotation_matrix
from pygltflib import GLTF2
import pyrender
import alphashape
import matplotlib.pyplot as plt
from shapely.geometry import Polygon
from scipy.optimize import least_squares

TODOS:
Load data: - Doneish
    Load Fit file as a list of dataframes - DONE
    Load metadata - Improve
        Replace hardcoded placeholder for mock metadata
        Generate a more elaborate metadata structure
    Load video - Elaborate
        Currently using a video key, maybe improve the way things are done
    Load full dataset - Improve
        Replace hardcoded strings to match the saving scheme
        Figure out the basic saving scheme for user data and metadata
        # TODO: get user key from connection request instead
        # TODO Later change to session date & time
    SessionData - Build up
        Improve SessionData to contain utility methods
        Rename SessionData as it means something slightly different in IT
        
Video correction - Doneish
    Load video - Improve
        Replace hardcoding with a dataset attribute
        Maybe save video to dataset as well?
        Or add the functions as utility for later caching?
    Load camera matrix - Implement
        Store the camera matrix properly in the metadata
        Currently using good enough mock matrix for correction (works suprisingly well)
        Add a fallback to using the default matrix if camera is uncallibrated
    Camera matrix - Build up
        Add helper methods to save and read from metadata
    Correct video - Clean
        Code works and did not have any notes, but the method is tad long and would benefit from a second look

Getting pose - Doneish
    Get pose - Elaborate & clean
        The method works okay for a prototype, but it has a few issues
        First, since the video is closed after calling it, it fails after executing once without reopening and recorrecting the video.
        This indicates that maybe the workflow should be improved in relation to working with video
        Also, did not test out visualization yet

Converting pose to a 3d points skeleton (To be fair, this should be - convert from third party pose to our 'simpified' model (33 3D point list)) - Implement conversion to 3D
    Unit vector extraction - Finish implementation and Test 
        ExtractCoords - Improve
            verify if using a list of pose_landmarks (point list) is the best way of doing this
        LoadCameraFrame - Implement
        PointsToUnitVectors - Improve
            camera frame is unused
    Conversion to 3D - Implement

Pose to area - Implement FitMesh from ChatGPT (This then is a method for getting area from pose. That said, in some cases pose could be skipped, but then two previous steps are just a tad different and this could be seen as video to 2d area shape (and this could differ based on the calculator used, as some sims may use a 3d surface for CdA) in the end, pose gets the riders
configuration, while pose to area combines the riders model with the configuration to generate an avatar (a digital twin) for CdA calculation purposes)
    Mesh import - Finish implementation and Test
        Create/Find mock rider mesh
        Get rider mesh - Test
    FitMesh - Implement
    Create wind planes - Elaborate
    Project to plane - Test & improve

Calculate results - Doneish, Waiting for testing
    Get area - Test & Improve
        Throw exception on multishape
    Calculate .. - Test & Improve math
        Get accurate coeficients (from metadata?)
    Calculate results - Test & Improve
        Zip metadata
        Create mock metadata for velocity
        Check how to best tie together fit data with video data (timestamps?)        

# Load data

In [2]:
class SessionData:
    def __init__(self, video_key, fit_data, metadata):
        self.video_key = video_key
        self.fit_data = fit_data
        self.metadata = metadata
        self.name = metadata['name']

In [3]:
def load_fit_file(file_name, file_path, backup_data=False):
    """
    Load a FIT file and return the data as a pandas DataFrame.
    """
    fitfile = FitFile(file_path+file_name)

    data = {}
    unique_msg = set()
    for msg in fitfile.get_messages():
        unique_msg.add(msg.name)

    for table_name in unique_msg:
        table_list = []
        for table in fitfile.get_messages(table_name):
            table_data = {}
            for field in table:
                table_data[field.name] = field.value
            table_list.append(table_data)
        data[table_name] = pandas.DataFrame(table_list)

    if backup_data:
        for table_name, df in data.items():
            file_name = f"{file_name}_{table_name}.csv"
            df.to_csv(file_name, index=False)
    return data

In [4]:
fit_data = load_fit_file("Freezing_Pain.fit", "data/raw/")

print(type(fit_data))
print(type(fit_data.keys()))
print(type(fit_data.values()))
print(type(fit_data['record']))
print(fit_data['record'].head(5))
print(fit_data['record'].sample(5))

<class 'dict'>
<class 'dict_keys'>
<class 'dict_values'>
<class 'pandas.core.frame.DataFrame'>
   altitude  cadence  distance  enhanced_altitude  enhanced_speed  heart_rate  \
0      -1.0       71       6.1               -1.0           0.000         122   
1      -1.0       71       9.3               -1.0           0.000         122   
2      -1.0       71      12.4               -1.0           3.150         122   
3      -3.4       78      19.0               -3.4           4.300         122   
4      -3.4       82      25.6               -3.4           4.875         123   

   position_lat  position_long  power  speed  temperature           timestamp  
0     675960302      282546511  205.0  0.000           11 2025-03-11 14:23:18  
1     675960302      282546511  249.0  0.000           11 2025-03-11 14:23:19  
2     675960302      282546511  214.0  3.150           11 2025-03-11 14:23:20  
3     675959824      282545628  200.0  4.300           11 2025-03-11 14:23:21  
4     675959288   

In [26]:
fitfile = FitFile('data/raw/Freezing_Pain.fit')

for msg in fitfile.get_messages():
    if(msg.name != 'record'):
        print(msg.name)

print(type(fitfile))
print(type(fitfile.messages))
print(type(fitfile.messages[0]))
print(type(fitfile.messages[0].fields))
print(type(fitfile.messages[0].fields[0]))
print(type(fitfile.get_messages('lap')))
print(type(fitfile.get_messages('record')))
print(type(fitfile.parse()))
print(fitfile.parse())

for lap in fitfile.get_messages('lap'):  # 'record' messages contain the actual data points
    # Each lap has multiple data fields
    lap_data = {}
    for data in lap:
        lap_data[data.name] = data.value

    print(lap_data)

# Iterate through all records and print data
for record in fitfile.get_messages('record'):  # 'record' messages contain the actual data points

    # Each record has multiple data fields
    record_data = {}
    for data in record:
        record_data[data.name] = data.value

    print(record_data)

file_id
file_creator
user_profile
event
lap
lap
lap
lap
lap
lap
lap
lap
lap
lap
event
session
activity
<class 'fitparse.base.FitFile'>
<class 'list'>
<class 'fitparse.records.DataMessage'>
<class 'list'>
<class 'fitparse.records.FieldData'>
<class 'generator'>
<class 'generator'>


AttributeError: 'NoneType' object has no attribute 'tell'

In [5]:
# Create mock data
# Create mock metadata
metadata = {
    'name': 'Freezing Pain',
    'description': 'A session of freezing pain',
    'date': '2023-10-01',
    'location': 'Unknown',
    'duration': 3600,
    'calories': 500,
}
df_proto = [metadata]
df = pandas.DataFrame(df_proto)
df.to_csv('data/raw/metadata.csv', index=False)

In [6]:
def load_metadata(user_key, data_path):
    """
    Load metadata from a CSV file.
    """
    metadata = pandas.read_csv(data_path + "metadata.csv" )#(user_key)
    return metadata

In [7]:
def get_user_key_meta_data(fit_data):
    """
    Get user key meta data from the database.
    """
    return fit_data["user_profile"]["friendly_name"]
    

In [8]:
def load_dataset(data_path):
    session_id = "Freezing_Pain" # TODO Later change to session date & time
    # load video key or full video
    video_key = data_path.split('/')[-1].split('.')[0]
    # load fit file
    fit_data = load_fit_file(session_id + ".fit", data_path)
    # load user metadata
    meta_data = load_metadata(get_user_key_meta_data(fit_data), data_path) # TODO: get user key from connection request instead
    # convert it into our dataset
    dataset = SessionData(video_key, fit_data, meta_data)
    return dataset

In [9]:
data_path = "data/raw/"

In [10]:
data_key = sys.argv[1]
# DB = db_pck.load_db(env_var_db_path)
# data_path = load_path(DB.fetch_path(data_key))
dataset = load_dataset(data_path)

In [11]:
print(type(dataset.video_key))
print(type(dataset.fit_data))
print(type(dataset.metadata))

<class 'str'>
<class 'dict'>
<class 'pandas.core.frame.DataFrame'>


# Fisheye correction

In [12]:
class CameraMatrix:
    def __init__(self, K, D):
        self.K = K
        self.D = D

    def project(self, points):
        # Dummy projection function
        return points

    def unproject(self, points):
        # Dummy unprojection function
        return points

In [13]:
def LoadCameraMatrix(data_path):
    K = np.array([
        [320.0,   0.0, 320.0],  # fx, 0, cx
        [  0.0, 320.0, 240.0],  # 0, fy, cy
        [  0.0,   0.0,   1.0]
    ])
    D = np.array([[-0.18], [0.03], [0.0], [0.0]])
    cam_matrix = CameraMatrix(K,D)
    return cam_matrix

In [14]:
def CorrectVideo(cap, camera_matrix, output_path="./output/output.mp4", show_output = False): #video
    K = camera_matrix.K
    D = camera_matrix.D
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fps = cap.get(cv2.CAP_PROP_FPS)
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    out = cv2.VideoWriter(output_path, fourcc, fps, (width, height))
    new_K = cv2.fisheye.estimateNewCameraMatrixForUndistortRectify(K, D, (width, height), None)
    map1, map2 = cv2.fisheye.initUndistortRectifyMap(K, D, np.eye(3), new_K, (width, height), cv2.CV_16SC2)
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        # Undistort the frame using the precomputed maps
        undistorted_frame = cv2.remap(frame, map1, map2, interpolation=cv2.INTER_LINEAR)

        if show_output:
        # Show the frame (optional)
            cv2.imshow("Undistorted Video", undistorted_frame)
            if cv2.waitKey(1) & 0xFF == ord('q'):
                break

        # Write to output file
        
        out.write(undistorted_frame)

    # Release resources
    cap.release()
    out.release()
    cv2.destroyAllWindows()
    c_video = cv2.VideoCapture(output_path)
    return c_video
    

In [ ]:
# Mediapose stopgap code
mp_pose = mp.solutions.pose
pose = mp_pose.Pose()
mp_drawing = mp.solutions.drawing_utils

I0000 00:00:1745262963.840445 1464493 gl_context_egl.cc:85] Successfully initialized EGL. Major : 1 Minor: 5
I0000 00:00:1745262963.857097 2673540 gl_context.cc:369] GL version: 3.1 (OpenGL ES 3.1 Mesa 24.2.8-1ubuntu1~24.04.1), renderer: D3D12 (NVIDIA GeForce RTX 4060)


W0000 00:00:1745262963.951697 2673525 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1745262964.002242 2673529 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.


In [17]:
video = cv2.VideoCapture(data_path + "video/"+ "cycle_demo.mp4")        #dataset.GetVideo()
camera_matrix = LoadCameraMatrix(data_path + "/camera_matrix") # dataset.meta.camera_matrix
c_video = CorrectVideo(video, camera_matrix)
print(c_video)

< cv2.VideoCapture 0x7f437d086330>


# Pose extraction

In [18]:
# TODO: add intermediate video capture
def GetPose(c_video, show_output=False,output_path="./output/output_pose.mp4"):
    result_list = []
    width = int(c_video.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(c_video.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fps = c_video.get(cv2.CAP_PROP_FPS)
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    out = cv2.VideoWriter(output_path, fourcc, fps, (width, height))
    while c_video.isOpened():
        ret, frame = c_video.read()
        if not ret:
            break

        # Convert the BGR image to RGB
        rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

        # Process the frame with MediaPipe Pose
        results = pose.process(rgb_frame)

        # Draw pose landmarks on the frame
        if results.pose_landmarks:
            mp_drawing.draw_landmarks(
                frame,
                results.pose_landmarks,
                mp_pose.POSE_CONNECTIONS,
                mp_drawing.DrawingSpec(color=(0, 255, 0), thickness=2, circle_radius=2),
                mp_drawing.DrawingSpec(color=(0, 0, 255), thickness=2, circle_radius=2),
            )
        
        result_list.append(results) #check if deep copy needed

        # Display the frame
        if show_output:
            cv2.imshow('MediaPipe Pose', frame)
            # Exit loop on pressing 'q'
            if cv2.waitKey(1) & 0xFF == ord('q'):
                break

    c_video.release()
    cv2.destroyAllWindows()
    return result_list

In [ ]:
# Mediapipe stopgap code
while video.isOpened():
    ret, frame = video.read()
    if not ret:
        break

    # Convert the BGR image to RGB
    rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

    # Process the frame with MediaPipe Pose
    results = pose.process(rgb_frame)

    # Draw pose landmarks on the frame
    if results.pose_landmarks:
        mp_drawing.draw_landmarks(
            frame,
            results.pose_landmarks,
            mp_pose.POSE_CONNECTIONS,
            mp_drawing.DrawingSpec(color=(0, 255, 0), thickness=2, circle_radius=2),
            mp_drawing.DrawingSpec(color=(0, 0, 255), thickness=2, circle_radius=2),
        )

    # Display the frame
    cv2.imshow('MediaPipe Pose', frame)

    # Exit loop on pressing 'q'
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

video.release()
cv2.destroyAllWindows()

In [19]:
pose_info = GetPose(c_video) #GetPose(c_video)
print(type(pose_info))
print(pose_info)


W0000 00:00:1745263095.341437 2673523 landmark_projection_calculator.cc:186] Using NORM_RECT without IMAGE_DIMENSIONS is only supported for the square ROI. Provide IMAGE_DIMENSIONS or use PROJECTION_MATRIX.


<class 'list'>
[<class 'mediapipe.python.solution_base.SolutionOutputs'>, <class 'mediapipe.python.solution_base.SolutionOutputs'>, <class 'mediapipe.python.solution_base.SolutionOutputs'>, <class 'mediapipe.python.solution_base.SolutionOutputs'>, <class 'mediapipe.python.solution_base.SolutionOutputs'>, <class 'mediapipe.python.solution_base.SolutionOutputs'>, <class 'mediapipe.python.solution_base.SolutionOutputs'>, <class 'mediapipe.python.solution_base.SolutionOutputs'>, <class 'mediapipe.python.solution_base.SolutionOutputs'>, <class 'mediapipe.python.solution_base.SolutionOutputs'>, <class 'mediapipe.python.solution_base.SolutionOutputs'>, <class 'mediapipe.python.solution_base.SolutionOutputs'>, <class 'mediapipe.python.solution_base.SolutionOutputs'>, <class 'mediapipe.python.solution_base.SolutionOutputs'>, <class 'mediapipe.python.solution_base.SolutionOutputs'>, <class 'mediapipe.python.solution_base.SolutionOutputs'>, <class 'mediapipe.python.solution_base.SolutionOutputs'>

In [20]:
print(type(pose_info[0]))
print(type(pose_info[0].pose_landmarks))
print(type(pose_info[0].pose_landmarks.landmark))
print(type(pose_info[0]))
print(pose_info[0].pose_landmarks.landmark)
print(type(pose_info[0].pose_landmarks.landmark))

<class 'type'>
<class 'mediapipe.framework.formats.landmark_pb2.NormalizedLandmarkList'>
<class 'google._upb._message.RepeatedCompositeContainer'>
<class 'type'>
[x: 0.422408581
y: 0.138015613
z: -0.486073583
visibility: 0.999927878
, x: 0.432492167
y: 0.0992877483
z: -0.473404974
visibility: 0.999850273
, x: 0.440512717
y: 0.0975428522
z: -0.473418623
visibility: 0.99981457
, x: 0.44799608
y: 0.0960707
z: -0.473492324
visibility: 0.999841094
, x: 0.399934143
y: 0.104568094
z: -0.475177944
visibility: 0.999726593
, x: 0.390276372
y: 0.106092006
z: -0.475209296
visibility: 0.999633074
, x: 0.382227451
y: 0.107128412
z: -0.475250244
visibility: 0.999619126
, x: 0.459539831
y: 0.101796716
z: -0.344902277
visibility: 0.999815762
, x: 0.37091431
y: 0.115304977
z: -0.356209606
visibility: 0.99919945
, x: 0.440815121
y: 0.159771234
z: -0.433554351
visibility: 0.999922395
, x: 0.412032306
y: 0.169854432
z: -0.436514467
visibility: 0.999881268
, x: 0.514167905
y: 0.219854921
z: -0.25668785
visi

# Unit vector extraction

In [21]:
def ExtractPointsFromLandmarks(pose):
    points = []
    try:
        for el in enumerate(pose.pose_landmarks.landmark):
            points.append([el[1].x, el[1].y])
    except:
        print(f"Error extracting points from landmarks")
        return np.zeros((33, 2))
    return np.array(points)

In [22]:
def ExtractPoints(pose_info):
    points2d = []
    err_counter = 0
    for frame in pose_info:
        points2d.append(ExtractPointsFromLandmarks(frame))
    return np.array(points2d)

In [23]:
def PointsToUnitVectors(points, camera_frame):
    ones = np.ones((points.shape[0], 1))
    direction_vectors = np.hstack((points.reshape(-1, 2), ones))
    direction_vectors /= np.linalg.norm(direction_vectors, axis=1, keepdims=True)
    return direction_vectors

In [24]:
def PointListToUnitVectors(frames, camera_frame):
    unit_vectors = []
    for frame in frames:
        unit_vectors.append(PointsToUnitVectors(frame, camera_frame))
    return np.array(unit_vectors)

In [25]:
def LoadCameraFrame(data_path): #loads a camera reference frame, used to convert local coordinates to global coordinates
    pass

In [26]:
points2D = ExtractPoints(pose_info)
camera_frame = LoadCameraFrame(data_path + "/camera_matrix") #dataset.meta.camera_frame
unit_vectors = PointListToUnitVectors(points2D, [])# dataset.meta.camera_frame)

Error extracting points from landmarks
Error extracting points from landmarks
Error extracting points from landmarks
Error extracting points from landmarks
Error extracting points from landmarks
Error extracting points from landmarks
Error extracting points from landmarks
Error extracting points from landmarks
Error extracting points from landmarks
Error extracting points from landmarks
Error extracting points from landmarks
Error extracting points from landmarks
Error extracting points from landmarks
Error extracting points from landmarks
Error extracting points from landmarks
Error extracting points from landmarks
Error extracting points from landmarks
Error extracting points from landmarks
Error extracting points from landmarks
Error extracting points from landmarks
Error extracting points from landmarks
Error extracting points from landmarks
Error extracting points from landmarks
Error extracting points from landmarks
Error extracting points from landmarks
Error extracting points f

# Conversion to 3D

In [27]:
def calculate_camera_distance(coords_start, coords_end, image_width=None, image_height=None, pixel_coords=False):
    """Compute Euclidean distance between two landmarks."""
    if pixel_coords and image_width and image_height:
        x1, y1 = coords_start.x * image_width, coords_start.y * image_height
        x2, y2 = coords_end.x * image_width, coords_end.y * image_height
    else:
        x1, y1 = coords_start.x, coords_start.y
        x2, y2 = coords_end.x, coords_end.y

    return math.sqrt((x2 - x1)**2 + (y2 - y1)**2)

In [76]:
def GetPoseSegments():
    POSE_SEGMENTS = {
    # Head and Torso
    "head_to_shoulders": (0, 11),       # Nose to Left Shoulder
    "shoulders": (11, 12),              # Left Shoulder to Right Shoulder
    "torso": (23, 24),                  # Left Hip to Right Hip
    "spine_left": (11, 23),            # Left Shoulder to Left Hip
    "spine_right": (12, 24),            # Right Shoulder to Right Hip

    # Left Arm
    "left_upper_arm": (11, 13),         # Shoulder to Elbow
    "left_forearm": (13, 15),           # Elbow to Wrist
    "left_hand_index": (15, 19),        # Wrist to Index Finger Tip
    "left_hand_thumb": (15, 21),        # Wrist to Thumb Tip

    # Right Arm
    "right_upper_arm": (12, 14),        # Shoulder to Elbow
    "right_forearm": (14, 16),          # Elbow to Wrist
    "right_hand_index": (16, 20),       # Wrist to Index Finger Tip
    "right_hand_thumb": (16, 22),       # Wrist to Thumb Tip

    # Left Leg
    "left_thigh": (23, 25),             # Hip to Knee
    "left_shin": (25, 27),              # Knee to Ankle
    "left_foot": (27, 31),              # Ankle to Foot Index (toe)

    # Right Leg
    "right_thigh": (24, 26),            # Hip to Knee
    "right_shin": (26, 28),             # Knee to Ankle
    "right_foot": (28, 32),             # Ankle to Foot Index (toe)
    }
    return POSE_SEGMENTS

In [77]:
def LeastSquares(unit_vectors, point_distance_pairs):
    POSE_SEGMENTS = GetPoseSegments()
    # calculate distances between point pairs
    distances = {}
    #for segment, (start, end) in POSE_SEGMENTS.items():
    #    distances[segment] = calculate_camera_distance(pose[start], pose[end])
    
    # least squares fitting
    def error_function(X):
        X = X.reshape(-1, 3)  # Reshape to (N, 3)
        errors = []
        for key, d_ij in point_distance_pairs.items():
            i,j  = POSE_SEGMENTS[key]
            dist = np.linalg.norm(X[i] - X[j])  # Euclidean distance
            errors.append(dist - d_ij)  # Difference from known distance
        return errors

    # Initial guess: Assume all points lie along their respective unit vectors, scaled
    initial_guess = unit_vectors * 2.0  # Arbitrary scale

    # Solve using least squares optimization
    result = least_squares(error_function, initial_guess.ravel())

    # Extract optimized 3D points
    reconstructed_points = result.x.reshape(-1, 3)
    #print("Reconstructed 3D Points:\n", reconstructed_points)

    return reconstructed_points

In [29]:
def GetConstPairs(data_path):
    # TODO get pairs from metadata
    pairs = {
    # Head and Torso
    "head_to_shoulders": 0.2,       # Nose to Left Shoulder
    "shoulders": 0.34,              # Left Shoulder to Right Shoulder
    "torso": 0.32,                  # Left Hip to Right Hip
    "spine_left": 0.50,            # Left Shoulder to Left Hip
    "spine_right": 0.52,            # Right Shoulder to Right Hip

    # Left Arm
    "left_upper_arm": 0.3,         # Shoulder to Elbow
    "left_forearm": 0.18,           # Elbow to Wrist
    "left_hand_index": 0.10,        # Wrist to Index Finger Tip
    "left_hand_thumb": 0.08,        # Wrist to Thumb Tip

    # Right Arm
    "right_upper_arm": 0.3,        # Shoulder to Elbow
    "right_forearm": 0.18,          # Elbow to Wrist
    "right_hand_index": 0.10,       # Wrist to Index Finger Tip
    "right_hand_thumb": 0.08,       # Wrist to Thumb Tip

    # Left Leg
    "left_thigh": 0.50,             # Hip to Knee
    "left_shin": 0.35,              # Knee to Ankle
    "left_foot": 0.2,              # Ankle to Foot Index (toe)

    # Right Leg
    "right_thigh": 0.5,            # Hip to Knee
    "right_shin": 0.35,             # Knee to Ankle
    "right_foot": 0.2,             # Ankle to Foot Index (toe)
    }

    return pairs

In [30]:
def LeastSquaresArray(unit_vector_frames, const_pairs):
    points3D_ = []
    for unit_vectors in unit_vector_frames:
        points3D_.append(LeastSquares(unit_vectors, const_pairs))
    return points3D_

In [31]:
const_pairs = GetConstPairs(data_path + "/rider_keypointpairs") #dataset.meta.rider_info.GetConstPairs()
points3D = LeastSquaresArray(unit_vectors, const_pairs)

In [32]:
print(type(points3D))
print(type(points3D[0]))
print(points3D[0].shape)
print(points3D[0])

<class 'list'>
<class 'numpy.ndarray'>
(33, 3)
[[0.73373793 0.19798252 1.85625191]
 [0.79063847 0.18150783 1.82809894]
 [0.80307014 0.17782404 1.82303509]
 [0.81456187 0.17467905 1.81823436]
 [0.73919998 0.19327365 1.84830427]
 [0.72361203 0.19670535 1.85410157]
 [0.71052183 0.19914079 1.85889797]
 [0.83157114 0.18420865 1.8095736 ]
 [0.69149644 0.21496334 1.8643024 ]
 [0.79824129 0.28931856 1.81083009]
 [0.75269719 0.31028867 1.82679169]
 [0.91263208 0.24262569 1.77876727]
 [0.60014375 0.20246426 1.90658784]
 [0.88302311 0.53266141 1.7080383 ]
 [0.45749278 0.46605196 1.91971113]
 [0.82602068 0.69756916 1.66380981]
 [0.48834123 0.60779774 1.81314422]
 [0.83131454 0.83694549 1.61506606]
 [0.40470379 0.74723724 1.81048373]
 [0.82894024 0.79374049 1.63656003]
 [0.45795948 0.70301495 1.8164049 ]
 [0.83318951 0.77449303 1.64304143]
 [0.4854202  0.68758164 1.81824286]
 [0.86143019 0.73926258 1.7517457 ]
 [0.58600821 0.72222247 1.91377153]
 [1.17863667 0.90860096 1.40432008]
 [0.55502912 1.15

# Points to Area

In [ ]:
def compute_bone_transform(p1, p2):
    """Returns rotation matrix to align bone with direction p1 → p2"""
    direction = np.array(p2) - np.array(p1)
    direction /= np.linalg.norm(direction)
    # Assume rest pose is along +Y, compute rotation to match
    rot, _ = R.align_vectors([direction], [[0, 1, 0]])
    return rot.as_matrix()

In [59]:
def Fit3DMesh(points, mesh):
    # points is a 33 x 3 3D point array
    # mesh is a gltf skeleton that can be rigged to pose
    # fitted mesh is the initial mesh fitted to the given pose
    #fitted_mesh = trimesh.load(mesh, process=False)
    fitted_mesh = mesh
    #for bone in fitted_mesh:
        # get the pair of points
        # Align the bone to mesh
        #pass
    return fitted_mesh

In [51]:
def GetRiderMesh(data_path):
    scene = trimesh.load(data_path, process=False)
    mesh = scene.dump(concatenate=True)
    return mesh

In [53]:
def GetRiderMeshFromGLTF(data_path):
    print(data_path)
    gltf = GLTF2().load(data_path)
    print(gltf)
    #mesh = trimesh.load(gltf, process=False)
    return gltf

In [111]:
def create_cylinder_between(p1, p2, radius=0.05, sections=16):
    p1, p2 = np.array(p1), np.array(p2)
    vec = p2 - p1
    height = np.linalg.norm(vec)

    # Create cylinder along Z-axis
    cylinder = trimesh.creation.cylinder(radius=radius, height=height, sections=sections)

    # Align it to the vector
    z_axis = np.array([0, 0, 1])
    if not np.allclose(vec, z_axis):
        transform = trimesh.geometry.align_vectors(z_axis, vec)
        cylinder.apply_transform(transform)

    # Move to midpoint
    midpoint = (p1 + p2) / 2
    cylinder.apply_translation(midpoint)

    return cylinder

In [109]:
def show_mesh(mesh, points, point_pairs):
    # Show the mesh using trimesh
    #mesh = mesh.dump(concatenate=True)
    scene = pyrender.Scene()
    mesh = pyrender.Mesh.from_trimesh(mesh)
    camera = pyrender.PerspectiveCamera(yfov=np.pi / 3.0)
    rot_z = rotation_matrix(np.radians(90), [0, 1, 0])

    # Optional: move camera back a bit to see the scene
    trans = np.eye(4)
    trans[:3, 3] = [0, 0, 2]  # Move camera 2 units back along Z

    # Combine rotation and translation
    camera_pose = trans @ rot_z

    # Add camera to scene
    scene.add(camera, pose=camera_pose)
    scene.add(mesh)
    for point in points:
        s = trimesh.creation.icosphere(radius=0.01)
        s.apply_translation(point)
        scene.add(pyrender.Mesh.from_trimesh(s))
    for p1,p2 in point_pairs:
        cylinder = create_cylinder_between(p1, p2)
        scene.add(pyrender.Mesh.from_trimesh(cylinder))
    
    pyrender.Viewer(scene, use_raymond_lighting=True)

In [ ]:
def GetRiderMeasurements(data_path):
    csv_data = pandas.read_csv(data_path)
    return csv_data

In [ ]:
def ScaleToRider(model, rider_measurements):
    # Scale joints in the model to match rider measurements
    scaled_model = model.copy()
    for i, row in rider_measurements.iterrows():
        # Assuming the first column is the name and the second is the measurement
        name = row[0]
        measurement = row[1]
        # Find the corresponding mesh part and scale it
        if name in model:
            model[name].vertices *= measurement
    return scaled_model

In [ ]:
def ConvertToTrimesh(gltf, points):
    # Convert GLTF to trimesh through saving and loading the mesh
    gltf.save("./data/raw/temp.gltf")
    mesh = trimesh.load("./data/raw/temp.gltf", process=False)
    return mesh

In [92]:
def GetPointPairs(points, point_pairs):
    # Get the pairs of points from the list
    pairs = []
    for pair in point_pairs:
        print(pair)
        p1 = points[point_pairs[pair][0]]
        p2 = points[point_pairs[pair][1]]
        pairs.append((p1, p2))
    return np.array(pairs)

In [113]:
point_pairs = GetPoseSegments() #dataset.meta.rider_info.GetConstPairs()
print(point_pairs)
point_pairs_3d = GetPointPairs(points3D[0], point_pairs)
print(point_pairs_3d)

{'head_to_shoulders': (0, 11), 'shoulders': (11, 12), 'torso': (23, 24), 'spine_left': (11, 23), 'spine_right': (12, 24), 'left_upper_arm': (11, 13), 'left_forearm': (13, 15), 'left_hand_index': (15, 19), 'left_hand_thumb': (15, 21), 'right_upper_arm': (12, 14), 'right_forearm': (14, 16), 'right_hand_index': (16, 20), 'right_hand_thumb': (16, 22), 'left_thigh': (23, 25), 'left_shin': (25, 27), 'left_foot': (27, 31), 'right_thigh': (24, 26), 'right_shin': (26, 28), 'right_foot': (28, 32)}
head_to_shoulders
shoulders
torso
spine_left
spine_right
left_upper_arm
left_forearm
left_hand_index
left_hand_thumb
right_upper_arm
right_forearm
right_hand_index
right_hand_thumb
left_thigh
left_shin
left_foot
right_thigh
right_shin
right_foot
[[[0.73373793 0.19798252 1.85625191]
  [0.91263208 0.24262569 1.77876727]]

 [[0.91263208 0.24262569 1.77876727]
  [0.60014375 0.20246426 1.90658784]]

 [[0.86143019 0.73926258 1.7517457 ]
  [0.58600821 0.72222247 1.91377153]]

 [[0.91263208 0.24262569 1.778767

In [114]:
mesh = GetRiderMesh("./" + data_path + "rider.gltf")

/tmp/ipykernel_1464493/897044725.py:3: DeprecationWarning: `Scene.dump(concatenate=True)` DEPRECATED FOR REMOVAL APRIL 2025: replace with `Scene.to_geometry()`
  mesh = scene.dump(concatenate=True)


In [115]:
#gltf = GetRiderMeshFromGLTF("./" + data_path + "rider.gltf") #dataset.meta.rider_info.GetMesh()
#sized_gltf = ScaleToRider(gltf, GetRiderMeasurements(data_path + "rider_measurements.csv")) #dataset.meta.rider_info.GetMeshMeasurements()
#fitted_gltf = Fit3DMesh(points3D[0], sized_gltf) # Needs to be converted to an array method or just used for a test. Maybe can be output as an animation?
#fitted_mesh = ConvertToTrimesh(fitted_gltf)
show_mesh(mesh, points3D[0], point_pairs_3d)

# Projection to Plane

In [116]:
def CreatePlane(normal_vec, camera_pos):
    return (normal_vec, camera_pos)

In [117]:
def CreatePlanes(wind_data, camera_position = (0,0,0)):
    planes = []
    for datapoint in wind_data:
        planes.append(CreatePlane(datapoint, camera_position)) # TODO Replace with a better method
    return planes

In [120]:
def ProjectToPlane(polyline, plane):
    polyline[:, 2] = 0  # Set Z-coordinates to 0
    return polyline

#def flatten_to_2d(points_3d, plane_point, plane_normal):
#    u, v = plane_basis(plane_normal)
#    points_2d = []
#    for p in points_3d:
#        vec = p - plane_point
#        x = np.dot(vec, u)
#        y = np.dot(vec, v)
#        points_2d.append((x, y))
#    return points_2d
    #normal = plane[0] / np.linalg.norm(plane[0])  # Normalize normal vector

    # Compute distances from points to the plane
    #d = np.dot(polyline - plane[1], normal) / np.dot(normal, normal)

    # Compute projected points
    #projected_points = polyline - d[:, np.newaxis] * normal

    #return projected_points

In [118]:
def ProjectToPlanes(mesh, points3D, planes):
    shapes = []
    default_plane = planes[0] # add a getter for default plane
    for pose in points3D:
        fitted_mesh = Fit3DMesh(pose, mesh)
        shape = ProjectToPlane(fitted_mesh.vertices, default_plane) #TODO replace default plane with an itterated plane from planes (requires zipping points 3d and planes)
        shapes.append(shape)
    return shapes

In [121]:
plane_origin = np.array([0, 0, 0])
plane_normal = np.array([0, 0, 1])
planes = [(plane_normal, plane_origin)]#CreatePlanes(dataset.wind_direction) # Needs to be converted to an array method
polyshapes = ProjectToPlanes(mesh, points3D, planes) # Needs to be converted to an array method # technically, can be gotten through a different projection as well - project keypoints and draw an outline via points and arcs

# Calculate CdA

In [122]:
def GetArea(polyshape, alpha = 0.3):
    alpha_shape = alphashape.alphashape(polyshape, alpha)
    if isinstance(alpha_shape, Polygon):  # Ensure it's a polygon, not a multipolygon
        area = alpha_shape.area
        print(f"Alpha Shape Area: {area:.2f} square units")
        return area
    else:
        print("Alpha shape resulted in multiple disconnected components")
        return 0 # later iterate through components

In [123]:
def CalculateCdA(area, coifs=[0.3]):
    return area*coifs[0]

In [124]:
def CalculateDrag(velocity, cda, air_density=1):
    return 0.5 * air_density * cda * velocity**2

In [125]:
def CalculateRelative(power, drag, in_per):
    return 100*drag/power

In [126]:
def CalculateParamsFromShapes(polyshapes, velocity_data, backup_results=False):
    outputs = []
    for polyshape in polyshapes:
        results = {}
        results['Area'] = GetArea(polyshape)
        results['Velocity'] = velocity_data #TODO Zip velocity data to polyshape
        results['CdA'] = CalculateCdA(results['Area'])
        results['Drag'] = CalculateDrag(results['Velocity'], results['CdA'])
        results['Power'] = dataset.fit_data['record']['power'] #TODO Zip power data to polyshape
        results['Relative'] = CalculateRelative(results['Power'], results['Drag'])
        outputs.append(results)
    df = pandas.DataFrame(outputs)
    if backup_results:
        df.to_csv('data/output/outputs.csv', index=False)
    return df

In [127]:
cda_results = CalculateParamsFromShapes(polyshapes, dataset.velocity)
print(cda_results.sample(5))

AttributeError: 'SessionData' object has no attribute 'velocity'

In [128]:
# Example calculation for results
area = GetArea(polyshapes[0])
CdA = CalculateCdA(area)
drag = CalculateDrag(dataset.velocity, CdA)
rel_power = CalculateRelative(dataset.power, drag)

QhullError: QH6154 Qhull precision error: Initial simplex is flat (facet 1 is coplanar with the interior point)

While executing:  | qhull d Qt Qz Qc Q12 Qbb
Options selected for Qhull 2019.1.r 2019/06/21:
  run-id 254511612  delaunay  Qtriangulate  Qz-infinity-point  Qcoplanar-keep
  Q12-allow-wide  Qbbound-last  _pre-merge  _zero-centrum  Qinterior-keep
  Pgood  _max-width 1.4  Error-roundoff 2.8e-15  _one-merge 2.6e-14
  Visible-distance 1.7e-14  U-max-coplanar 1.7e-14  Width-outside 3.4e-14
  _wide-facet 1e-13  _maxoutside 3.4e-14

precision problems (corrected unless 'Q0' or an error)
  12007 zero divisors during gaussian elimination

The input to qhull appears to be less than 4 dimensional, or a
computation has overflowed.

Qhull could not construct a clearly convex simplex from points:
- p0(v5): -0.12 0.081     0 0.0033
- p6922(v4):  0.15 -0.0046     0 0.0037
- p9041(v3): -2.3e-05   1.4     0   1.3
- p11702(v2):  0.37  0.64     0  0.34
- p5605(v1): -0.37  0.64     0  0.34

The c

# Realistic optimization

It is impossible for the rider to reduce drag to zero, however it is possible for the rider to optimize their posture.
To find these moments, we must find the minimum CdA that the rider can realistically achieve based on the calculated losses.

In [ ]:
def GetMinimum(cda_results):
    return cda_results.min()

In [ ]:
cda_min = GetMinimum(cda_results['CdA'])
df = pandas.DataFrame(cda_results['CdA'])
df['delta'] = df['CdA'] - cda_min

# Visualize

To visualize the results, display the calculated params as graphs.

Visualize the power breakdown as well to illustrate how much of the total power is drag, and how much of that drag can be avoided by improved posture.

In [ ]:
#play_video(c_video)
#overlay(pose_info)
#overlay(polyshape)

#draw3D(points3D)
#draw3D(polyline)
#draw2D(polyshape)

#drawLabel(area, CdA, drag, rel_power, dataset.velocity, dataset.power)
#drawGraphs(...)